<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/009_PDF_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title PDF結合＆分割アプリ

# 必要ライブラリのインストール
!pip install -q PyPDF2

# ===== ライブラリインポート =====
import os
from PyPDF2 import PdfReader, PdfWriter
from google.colab import files
from IPython.display import display, HTML
from google.colab import output

# ===== メインUIの表示 =====
def display_ui():
    display(HTML('''
        <h2>📄 PDFツール（結合＆分割）</h2>
        <p>実行したい処理を選んでください。</p>
        <button onclick="google.colab.kernel.invokeFunction('merge_pdfs', [], {})"
                style="background-color:#4CAF50;color:white;padding:10px 20px;margin:10px;border:none;border-radius:5px;">PDF結合</button>
        <button onclick="google.colab.kernel.invokeFunction('split_pdf', [], {})"
                style="background-color:#2196F3;color:white;padding:10px 20px;margin:10px;border:none;border-radius:5px;">PDF分割</button>
    '''))

# ===== PDF結合処理 =====
def merge_pdfs_ui():
    print("✅ 結合したいPDFファイルをすべてアップロードしてください（順番に）：")
    uploaded_files = files.upload()
    pdf_writer = PdfWriter()

    # アップロード順に処理
    for filename in uploaded_files.keys():
        reader = PdfReader(filename)
        for page in reader.pages:
            pdf_writer.add_page(page)

    output_filename = "merged.pdf"
    with open(output_filename, "wb") as f:
        pdf_writer.write(f)

    print(f"\n🎉 結合完了！ -> {output_filename}")
    files.download(output_filename)

# ===== PDF分割処理 =====
def split_pdf_ui():
    print("📥 分割したいPDFファイルを1つアップロードしてください：")
    uploaded_file = files.upload()
    filename = list(uploaded_file.keys())[0]
    reader = PdfReader(filename)

    for i, page in enumerate(reader.pages):
        writer = PdfWriter()
        writer.add_page(page)
        output_filename = f"page_{i+1}.pdf"
        with open(output_filename, "wb") as f:
            writer.write(f)
        files.download(output_filename)
        print(f"✅ {output_filename} を出力しました")

# ===== コールバック登録（ボタンと処理を紐づけ） =====
output.register_callback('merge_pdfs', merge_pdfs_ui)
output.register_callback('split_pdf', split_pdf_ui)

# UI表示
display_ui()


In [ ]:
# インストール（初回のみ）
!pip install -q PyPDF2 pdf2image
!apt-get -y install poppler-utils

import os
import base64
import uuid
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_path
from google.colab import files
from IPython.display import display, HTML, Javascript
from google.colab import output

os.makedirs("pdf_inputs", exist_ok=True)
os.makedirs("thumbnails", exist_ok=True)
uploaded_files = {}

def upload_pdfs():
    print("📥 PDFをアップロードしてください：")
    uploaded = files.upload()
    for name in uploaded.keys():
        path = f"pdf_inputs/{name}"
        os.rename(name, path)
        uploaded_files[name] = path
    generate_thumbnails()

def generate_thumbnails():
    html = '''
    <h3>🖼 サムネイルを並べ替えてください（ドラッグ＆ドロップ）</h3>
    <ul id="thumbList" style="list-style:none; display:flex; flex-wrap:wrap; gap:10px;">
    '''
    for i, (name, path) in enumerate(uploaded_files.items()):
        image = convert_from_path(path, first_page=1, last_page=1)[0]
        img_path = f"thumbnails/{uuid.uuid4().hex}.jpg"
        image.save(img_path, "JPEG")
        with open(img_path, "rb") as img_file:
            b64 = base64.b64encode(img_file.read()).decode('utf-8')
        html += f'''
        <li draggable="true" class="thumbItem" data-filename="{name}" style="cursor:move;">
          <img src="data:image/jpeg;base64,{b64}" width="120"><br><center>{name}</center>
        </li>
        '''
    html += '''
    </ul>
    <button onclick="getSortedOrder()" style="margin-top:20px;padding:10px 20px;">✅ この順で結合</button>
    <script>
      let dragging;

      document.querySelectorAll('.thumbItem').forEach(item => {
        item.addEventListener('dragstart', e => dragging = item);
        item.addEventListener('dragover', e => e.preventDefault());
        item.addEventListener('drop', e => {
          e.preventDefault();
          if (dragging && dragging !== e.target.closest('.thumbItem')) {
            const list = dragging.parentNode;
            const dropItem = e.target.closest('.thumbItem');
            list.insertBefore(dragging, dropItem);
          }
        });
      });

      function getSortedOrder() {
        const items = document.querySelectorAll('.thumbItem');
        const filenames = [...items].map(el => el.dataset.filename);
        google.colab.kernel.invokeFunction('merge_sorted_order', [filenames], {});
      }
    </script>
    '''
    display(HTML(html))

def merge_sorted_order(filenames):
    writer = PdfWriter()
    for name in filenames:
        path = uploaded_files[name]
        reader = PdfReader(path)
        for page in reader.pages:
            writer.add_page(page)
    output_path = "merged_sorted.pdf"
    with open(output_path, "wb") as f:
        writer.write(f)
    print("✅ 結合完了！")
    files.download(output_path)

# 登録と実行
output.register_callback('merge_sorted_order', merge_sorted_order)
upload_pdfs()


# 📄 PDF結合＆分割アプリ 仕様書

## 1. 概要

本アプリケーションは、Google Colab上で動作するPythonベースのツールであり、ユーザーがアップロードしたPDFファイルを結合（マージ）または分割（スプリット）することができます。GUI風の簡易インターフェースにより、非エンジニアユーザーでも直感的に操作できることを目的とします。

---

## 2. 対象ユーザー

- PDFの編集を手軽に行いたい一般ユーザー
- エンジニアや研究者（プログラム不要で操作できる）
- ブラウザベース（Google Colab）で利用したいユーザー

---

## 3. 利用環境

- **プラットフォーム**：Google Colab（ブラウザ上）
- **使用言語**：Python 3.x
- **必要ライブラリ**：
  - `PyPDF2`（PDF操作）
  - `google.colab`（ファイルアップロード・ダウンロード用）
  - `IPython.display`（UI表示）

---

## 4. 機能一覧

### 4.1 PDF結合機能（Merge PDFs）

#### 説明：
複数のPDFファイルをアップロードし、1つのPDFに結合します。

#### 操作手順：
1. 「PDF結合」ボタンをクリック
2. 結合したいPDFをすべてアップロード（順番に）
3. 自動的に結合し、「merged.pdf」としてダウンロードリンクを表示

#### 備考：
- 結合順はアップロード順に依存します
- 出力ファイル名は固定で `merged.pdf`

---

### 4.2 PDF分割機能（Split PDF）

#### 説明：
1つのPDFファイルをページ単位に分割し、各ページを個別のPDFとして出力します。

#### 操作手順：
1. 「PDF分割」ボタンをクリック
2. 分割対象のPDFを1つアップロード
3. ページごとに分割して、自動ダウンロード（例：`page_1.pdf`, `page_2.pdf`, ...）

#### 備考：
- 現時点では「ページ範囲指定」は未対応（全ページ分割）
- ダウンロードは1ページごとに順次行われる

---

## 5. ファイル管理

- 一時的なファイルはColabの仮想ファイルシステム（セッション内）に保存され、セッション終了時に消去される。
- 出力ファイルはユーザー側で明示的にダウンロード。

---

## 6. 将来的な拡張予定（オプション）

以下の機能は要望に応じて追加可能：

| 機能                         | 説明 |
|------------------------------|------|
| ページ順の並べ替え           | 結合前にドラッグなどで順序を変更 |
| ページ範囲での分割           | 例：3～5ページ目だけ切り出す |
| サムネイル表示               | PDFページを小さくプレビュー |
| パスワード付きPDFの解除     | ロック付きPDFを処理可能にする（要PyPDF2の制限確認） |
| 圧縮（軽量化）               | 出力PDFのサイズを軽くする |
| 結合後PDFのページ削除／選択 | 不要ページを除外 |

---

## 7. 制約事項

- PyPDF2は一部のPDF（特殊なエンコードや保護付き）に対応していない場合あり
- 大容量PDFの処理には時間がかかる可能性がある
- ページごとの画像・フォント情報は保持されるが、正確な再現性はPDFの構造に依存

---

## 8. 著作権・ライセンス

- 本アプリはMITライセンス等の制限なしの形で配布予定（個人利用・教育目的で自由に使用可能）
- 使用ライブラリのライセンスには準拠

---

必要に応じて、これをMarkdownやPDF形式にして出力することも可能です。  
また、「ページ範囲指定」など特定機能を先に実装したい場合もお気軽にお知らせください！